# 네이버 카페 크롤러

## 필요 라이브러리 import

In [1]:
!pip install selenium

In [1]:
# selenium 셋팅

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [2]:
import time
import csv
import pandas as pd
import random
from bs4 import BeautifulSoup as bs

## 네이버 로그인

자동화된 크롬창을 실행시켜서 네이버 로그인 페이지 접속

1. 크롬드라이버 제어(크롬창 켜기)
2. 네이버 로그인(아이디 입력 -> 비밀번호 입력 -> 로그인 버튼 클릭 순서로 작업 진행)

In [3]:
# 크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox') 
chrome_options.add_argument('--disable-dex-shm-usage')

# 현재 컴큐터 크롬 드라이버 위치
chorme_path = 'chromedriver.exe' #본인의 크롬 드라이버 위치 입력

# 자동화된 크롬 창 실행
driver = webdriver.Chrome(executable_path=chorme_path, chrome_options=chrome_options)

# 네이버 로그인 페이지 접속
login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
driver.get(login_url)
driver.implicitly_wait(10)

# 본인의 아이디, 비밀번호를 각 변수에 저장
my_id = 'ddbb55'
my_pw = 'wheksqleoqkr'

# 아이디, 비밀번호 입력 (네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 사용)
#네이버에서 send_key()를 쓰면 로그인에서 자동입력방지기능이 나타남.
driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
time.sleep(1)

#'로그인' 버튼 클릭
driver.find_element('id','log.login').click()
time.sleep(1)

C:\Users\USER2\AppData\Local\Temp\ipykernel_6672\3778175403.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chorme_path, chrome_options=chrome_options)
C:\Users\USER2\AppData\Local\Temp\ipykernel_6672\3778175403.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chorme_path, chrome_options=chrome_options)


# 크롤링 할 카페 접속 및 원하는 게시판 클릭


1. 카페 접속
- 로그인한 계정이 카페에 회원가입/등업조건 만족해야 합니다.
2. 원하는 게시판 선택(클릭)
- F12(개발자 도구) 눌러 크롤링 대상 게시판의 선택자를 확인

In [4]:
# 카페 접속
time.sleep(2)
moms_url = 'https://cafe.naver.com/workee' #타겟 카페가 바뀔 경우 url을 바꿔주세요
driver.get(moms_url)
time.sleep(1)

# 원하는 게시판 클릭
driver.find_element('xpath','//*[@id="menuLink78"]').click() #게시판의 선택자 넣어주세요
time.sleep(1)

# 프레임 전환
driver.switch_to.frame('cafe_main')
time.sleep(1)

# 게시물 크롤링

1. 첫번째 게시물 클릭 - XPath 활용
2. 크롤링할 개수 지정
3. 크롤링

In [5]:
# 제목, 본문, 댓글, 시간 빈 list 생성

titles = [] #제목
reviews = [] #리뷰
comments = [] #댓글
date = [] # 시간

count_view=0


# 1. 첫번째 게시물 클릭 (선택자 입력_여기서는 xpath copy 해줌)
driver.find_element('xpath','//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div/a[1]').click()
#driver.find_element('xpath','//*[@id="main-area"]/div[5]/table/tbody/tr[5]/td[1]/div[2]/div/a[1]').click()


# 2. for 문으로 지정한 게시물 개수까지 돌림

for i in range(1): #새로 창열기
    for i in range(30): # 한 번에 몇 개의 개시물을 크롤링 할 것인지 지정        
        driver.implicitly_wait(30) 
        
        # 제목 수집
        time.sleep(random.uniform(0.5,1)) # 0.5~1초 랜덤한 시간으로 쉬어줘
        title = driver.find_element('xpath','//*[@id="app"]/div/div/div[2]/div[1]/div[1]/div/h3').text
        titles.append(title)    

       
        # 글 본문 수집 (3가지 경우 존재)
        # 본문의 경우 선택자 1을 확인하고 코드를 작성했으나, 또 다른 선택자가 존재하여 선택자 2를 다시 try해주도록 만들었습니다.
        try: 
            content = driver.find_element(By.CSS_SELECTOR,'div.se-module.se-module-text').text # 선택자1 ('div.se-module.se-module-text')
            reviews.append(content)       
            
        except:
            try:
                content = driver.find_element(By.CSS_SELECTOR,'div.ContentRenderer').text # 선택자2 ('div.ContentRenderer')
                reviews.append(content)
            except:
                content= [] # 본문이 사진으로만 구성 (담을게 없어서 빈 리스트 생성해서 append 시켜줌)
                reviews.append(content)


         # 댓글 수집 (2가지 경우 존재)
        soup = bs(driver.page_source, 'lxml') # 현재 페이지 HTML 정보 가지고 오기
        iscomment = soup.find_all('span',class_='text_comment') # find_all = text_comment 전부 다 찾아줘
        if len(iscomment) == 0: # 댓글 없는 경우
            box = []

        else: # 댓글 있는 경우
                
            WebDriverWait(driver, 15).until( EC.presence_of_all_elements_located((By.CLASS_NAME, "text_comment")) )
                # 웹드라이버를 통해 브라우져에서 최대 15초 기다려주고, text_comment(댓글 선택자) 기준으로 나올때 까지 기다려줘
            
            soup = bs(driver.page_source, 'lxml')  
            iscomment = soup.find_all('span',class_='text_comment') 
            box = []
            for i in iscomment:
                box.append([i.get_text()]) # iscome에 할당한 text_comment에서 text만 box에 append

        comments.append(box)

        # 시간 수집
        times = driver.find_element('xpath','//*[@id="app"]/div/div/div[2]/div[1]/div[2]/div/div[2]/span[1]').text
        date.append(times) 
        
        
        # count view 크롤링 진행상황 보기 (50개 마다 출력)
        count_view = count_view + 1
        if count_view % 50 ==0: #50으로 나누었을 때 0이 되면 게시물 크롤링 완료라고 보여줘
            print('{}개 게시물 크롤링 완료'.format(count_view))
        else:
            pass
        

        
        
        # 다음글 클릭 (다음글 버튼 클릭해서 연속적으로 크롤링, 첫번째글->두번째글->세번째글...)
        try:
            driver.find_element(By.CSS_SELECTOR,"#app > div > div > div.ArticleTopBtns > div.right_area > a.BaseButton.btn_next.BaseButton--skinGray.size_default > span").click()
            driver.implicitly_wait(28)
            
        except: # 다음글 버튼 사라지는 문제점 발생, 크롬 close 후 다시 open
            
            current_url = driver.current_url # 다음글 버튼이 사라져서 크롤링이 멈춘 url 가지고 오기
            driver.close()
            
            # 크롬창 다시 열기
            driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)  


            ##재로그인
            # 네이버 로그인 화면 이동
            login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
            driver.get(login_url)
            driver.implicitly_wait(9)

            driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
            driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
            time.sleep(1)


            # '로그인' 버튼 클릭
            driver.find_element('id','log.login').click()
            time.sleep(1)

            # 멈친 부분의 링크 가지고오기
            driver.get(current_url)
            driver.switch_to.frame('cafe_main')
            time.sleep(4)    

                  
    # 크롬이 다시 열릴 때 가지고 올 다음 링크 가져오기 #다음글 클릭에 해당되는 href 추출 (url로 이동해야 하기 때문에)
    link = driver.find_element('xpath','//*[@id="app"]/div/div/div[1]/div[2]/a[2]').get_attribute("href")
    
    #현재 크롬창 닫기
    driver.close()
    
    
    # 크롬창 다시 열기
    driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)  
    
       
    ##재로그인
    # 네이버 로그인 화면 이동
    login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
    driver.get(login_url)
    driver.implicitly_wait(9)

    driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
    driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
    time.sleep(1)

    # '로그인' 버튼 클릭
    driver.find_element('id','log.login').click()
    time.sleep(1)
    
    #다음 링크부터 가져오기
    driver.get(link)
    driver.switch_to.frame('cafe_main')


# 크롬 창 닫기
driver.quit()
        

KeyboardInterrupt: 

In [6]:
# 제목 리뷰 댓글이 잘 크롤링 되었는지 확인_게시물 개수를 의미함

print(len(titles))
print(len(reviews))
print(len(comments))
print(len(date))

6
6
6
6


In [8]:
reviews

['건설업 경력으로 입사한지 4개월 정도 됬어요.\n사무실에는 저, 밑에 직원(1년차), 부장님(사장님동생)이 계시는데\n1년차 직원이 일을 잘 못해서 너무 힘드네요 ㅠㅠ\n부장님-> 저 -> 1년차직원 이런식으로 일시켜서 제가 검토하고 보고 올리고\n사장님도 일시키고 너무 스트레스 받아요\n부장님도 직원분이 일못하는거 알고 있는데\n저한테 일시킬테니 저보고 그 직원 일 시켜서 하라고 하네요....\nㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ',
 '안녕하세요 맛 점 들 하셨나요?\n간단히 저를 소개하자면 국제물류주선업( 포워딩 ) 물류 업계에서 재직 중인 28살 남자입니다. 현재 3년 차이고요\n4개월 전에 동일 직종으로 이직하였습니다, 회사는 집에서 도보로 10분 거리이며 같이 일하는 사람들도 무척 좋고 연봉도 많이 올랐습니다.(이직 전 세후 180 이직 후 세후 280)\n최근 들어 업계가 힘들어서 그런지 일이 너무 없어요.. 9시 출근하면 2시간 만에 당일 업무 끝나는 경우 허다하여\n나머지 시간에는 영어 공부/직탐 등을 하며 시간 보내고 있습니다. 개인적으로 연봉에 맞지 않게 업무가 너무 적고 쉽다고 생각하여 계속 다니다간 물경력 되기에 적합하다고 생각합니다.\n이직 후 초반에는 월급 루팡이 너무 좋았는데 점점 하루하루 지나니 일이 없어 허송세월 하는 자체가 너무 힘듭니다. 과감하게 이직을 생각하고 있는데 또 이런 좋은 조건의 회사 찾기도 어려울 것 같고 근무 시간에 개인 공부를 하자니 집중도 잘 안되고 해서요.. 어떻게 하면 좋을지 형 누님들 한마디씩 조언 부탁드립니다.\n1. 과감 한 결단 (이직할 회사 찾기)\n2. 버티다 자격증 공부 더하여 이직\n3. 1년 버텨 보고 그때 다시 고민\n4. 4년제 학사 취득 (무역/물류 계열) / 제가 전문 학사 출신입니다.\n5. 기타 조언 편히 말씀 부탁드립니다.\n ',
 '현재 가족들과 함께 완공이 채 1년도 되지않은\n 신축아파트 거주중인데요\n솔직히 신축아파트에 살다보니 커뮤니티 시설도 최상이라 자취하고 싶은 마음도 없었

In [42]:
#===데이터 합치기(크롤링한 데이터를 하나의 dataframe 형태로 바꿔줌)

data = {'time' : date,
       'tit' : titles,
       'review' : reviews,
       'comment': comments
       }

dataDF =pd.DataFrame(data)

In [43]:
dataDF

,time,tit,review,comment
0,2022.11.10. 14:07,마니또 선물 뭐가 좋을까요?,연말이라고 회식도 하고\n제비뽑기로 마니또 지정해서 5만원이상 선물해줘야 하거든요 ...,[[니트나 히트텍?]]
1,2022.11.10. 14:03,직장 동료와 인스타 공유,어쩌다보니 직장 동료들과 인스타 맞팔을 하게되었는데요.. \n올리고 싶은 게시물은 ...,"[[탈퇴후 재 가입? 방법이 없을듯해요 ㅜㅜ], [그런가요ㅠㅠ 방법들을 많이 생각해..."
2,2022.11.10. 13:40,우째뿌까요?,다니는 곳은 교육기관이고 저는 비정규직임돠.\n행정실 공뭔이 내가 올린 품의 처리를...,"[[기한을 딱 정해주시고 그 기한까지 기다렸다가 넘기면 장감님께 말씀드리세요], [..."
3,2022.11.10. 13:28,실수를 해도 덤덤해요,저 왜 이제 실수를 해도 좀 덤덤하죠? ㅠㅠ \n신입때 막 실수하면 어쩌지어쩌지 잠...,[[수습 할 수 있는 능력도 생기고 가능하니.. 그러지 못한 실수라면.. 그런 생각...
4,2022.11.10. 11:51,가 조ㅈ회사..........,영업관리직으로 여기저기 옮겨다니며 일하다가\n한곳에 안정적으로 정착하고 싶어\n집에...,"[[역시 가적회사는 지멋대로 개판오분전. 힘내요. 마음의 상처 받지 말구, 쌩까면 ..."
5,2022.11.10. 11:20,월급에 대해 현타가 와요,"중소기업이고, 처음 입사 때 말도 안되는 금액으로\n입사를 했는데 이제 6년차예용....","[[일당 15만원 이하인가요??], [그런거같아요,.....ㅠㅠㅠ 아마도 계산은 안..."
6,2022.11.10. 10:58,"네일아트, 속눈썹연장 학원비 얼마정도 할까요??",동생이 너무 노답인생으로 살고 있어서\n꾸미는거에 관심이 많고 샵이나 카페 같은거 ...,"[[재료비 ,학원비 이런것까지 꽤 드는 것 같더라고요.], [학원마다 다른거같아요...."
7,2022.11.10. 09:22,장단점 극명한 회사,저는 지금 본가에 살고있어서\n2년 넘게 다니던 회사 퇴사하고\n지금 회사로 이직을...,[['다니면서 알아본다.'지금 회사가 오래 갈 수 있을까요? 중장기적으로 봤을 때 ...
8,2022.11.10. 09:03,이직 실패일까요... 요새 밤새 고민만하네요,간단히 쓰자면 공단쪽에서 계속 사무직 했었다가\n현장 일 같이 하면서 월급은 최저임...,"[[다 알면서 왜 들어가신다음에 고민을 하는건지..모르겠습니다], [2222], [..."
9,2022.11.10. 09:02,회사에 건달?같은 타부서 상사랑 싸워야할까요??,타부서에 성격 진짜 불같고 공격적인 상사가 있는데\n요즘 일이 조금 얽혀서 같이 일...,"[[싸우세여 \n\n], [상대 눈 똑바로 바라보면서 정색하고 말 하세요], [할말..."


In [44]:
#csv로 저장
dataDF.to_csv("직장인탐구생활.csv", encoding = "utf-8-sig") # 저장할 파일명.csv 입력 후 셀 실행